Per entrar a una sessió de Spark i iniciar un builder en DeltaLake

In [ ]:
#!pip install pyspark
#!pip install delta-spark

import pyspark
from delta import *
from pyspark.sql.functions import col, lit, pi, atan, exp
from uszipcode import SearchEngine
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType,  IntegerType
from geopy.geocoders import Nominatim
from outliers_data_Q import *



builder = pyspark.sql.SparkSession.builder.appName("Shops_Deltalake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

## Comandes com si estessis a una sessió de Spark

Llegir els arxius i guardar-los en format DeltaLake, que ens garantitza que es compleixen les restriccions ACID, i dona avanatatges 




Carregar i llegir els 3 arxius de tenim

In [6]:
shops = spark.read.parquet("./datalake/shops_data/2024-04-17_shops_data.parquet")
shops.show()

+-------------------+------------------+--------------------+-------------------------+---------------------------+----------------------+--------------------+---------------------+------------------------+------------------------+-----------------------+------------------------+---------------------------+--------------------+------------------+----------------+-------------------+--------------------+-------------------+---------------+--------------------+------------------+
|attributes.objectid|attributes.osm_id2|attributes.abandoned|attributes.addr_housename|attributes.addr_housenumber|attributes.addr_street|attributes.addr_city|attributes.addr_state|attributes.addr_postcode|attributes.addr_province|attributes.addr_country|attributes.addr_district|attributes.addr_subdistrict|attributes.addr_unit|attributes.amenity|attributes.brand|attributes.building|     attributes.name|attributes.operator|attributes.shop|          geometry.x|        geometry.y|
+-------------------+-------------

In [7]:
#Arxiu Parquet 
shops = spark.read.parquet("./datalake/shops_data/2024-04-17_shops_data.parquet")

shops = shops.drop('attributes.osm_id2')\
                .drop('attributes.abandoned')\
                .drop('attributes.addr_housename')\
                .drop('attributes.addr_housenumber')\
                .drop('attributes.addr_street')\
                .drop('attributes.addr_city')\
                .drop('attributes.addr_state')\
                .drop('attributes.addr_province')\
                .drop('attributes.addr_country')\
                .drop('attributes.addr_district')\
                .drop('attributes.addr_subdistrict')\
                .drop('attributes.addr_unit')\
                .drop('attributes.amenity')\
                .drop('attributes.brand')\
                .drop('attributes.building')\
                .drop('attributes.operator')

shops = shops.withColumnRenamed('attributes.objectid', 'Id')\
            .withColumnRenamed('attributes.addr_postcode', 'Zipcode')\
            .withColumnRenamed('attributes.name', 'Name')\
            .withColumnRenamed('attributes.shop', 'Shop Type')\
            .withColumnRenamed('geometry.x', 'Coordinate X')\
            .withColumnRenamed('geometry.y', 'Coordinate Y')

R = 6378137 #radi de la terra
shops = shops.withColumn('Longitude', (col('Coordinate X') / lit(R)) * (180 / pi()))\
            .withColumn('Latitude',  (((pi() / 2) - 2 * atan(exp(-col('Coordinate Y') / lit(R)))) * (180 / pi())))\
            .drop('Coordinate X')\
            .drop('Coordinate Y')

shops.show()

geolocator = Nominatim(user_agent="myGeocoder")

def impute_name(latitude, longitude):
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    result = location.address.split(',', 1)[0]
    if result:return result
    else: return None


impute_income_udf = udf(impute_name, IntegerType())
shops = shops.withColumn('Name', impute_income_udf( col('Latitude'), col('Longitude')))

shops.show()


+---+-------+--------------------+-------------+-------------------+------------------+
| Id|Zipcode|                Name|    Shop Type|          Longitude|          Latitude|
+---+-------+--------------------+-------------+-------------------+------------------+
| 85|   NULL|Jo Jo’s Clubhouse...|confectionery|       -159.6701292|        21.9560838|
| 95|   NULL|  The Shrimp Station|      seafood| -159.6681456997286|21.955308000073515|
| 98|   NULL|Big Save Super Ma...|  supermarket|-159.66764739974894|21.955425700504914|
|105|   NULL|                NULL|      laundry|-159.67449000021873| 21.95798440042452|
|107|   NULL|            Red Dirt|      clothes|       -159.6703658|21.956209899999998|
|367|   NULL|     Maile Tropicals|    cosmetics|       -159.4694964|        21.8835189|
|384|   NULL|Whalers General S...|  convenience|-159.45869900000002|21.878917499999996|
|399|   NULL|                NULL|      clothes|-159.45903810020474|21.879525500439332|
|400|   NULL|                NUL

Py4JJavaError: An error occurred while calling o152.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 6) (Paula executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2$a

In [ ]:
shops.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Name: integer (nullable = true)
 |-- Shop Type: string (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Latitude: double (nullable = true)



In [ ]:
categorical_columns = ["Zipcode", "Shop Type"]
numerical_columns = ["Id", "Name", "Longitude", "Latitude"]

In [ ]:
plots_cate(shops, categorical_columns)

In [ ]:
plots_num(shops, numerical_columns)

In [110]:
#### FORMA 1 DE FER-HO ###

from uszipcode import SearchEngine
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Crea un objeto SearchEngine
search = SearchEngine()

# Función para obtener el código postal
def get_zipcode(lat, lon):
    search = SearchEngine(simple_zipcode=True)  # Considerar usar simple_zipcode para mejorar la eficiencia
    result = search.by_coordinates(lat=lat, lng=lon, returns=1)
    if result:
        return result[0].zipcode
    else:
        return None

zipcode_udf = udf(get_zipcode, StringType())
shops = shops.withColumn('Zip Code', zipcode_udf(col('Latitude'), col('Longitude')))


In [95]:
### FORMA 2 DE FER-HO  ### 

def impute_income(latitude, longitude):
    search = SearchEngine(simple_zipcode=True)  # Considerar usar simple_zipcode para mejorar la eficiencia
    result = search.by_coordinates(lat=latitude, lng=longitude, returns=1)
    if result:
        return result[0].zipcode
    else:
        return None
# Registrar UDF
impute_income_udf = udf(impute_income, IntegerType())

# Aplicar UDF
shops = shops.withColumn('Zipcode', impute_income_udf( 'Latitude', 'Longitude'))

In [6]:
#Guardar l'arxiu
shops.write.mode("overwrite").format("delta").save("./deltalake/shops_data/")

Py4JJavaError: An error occurred while calling o48.save.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at io.delta.storage.HadoopFileSystemLogStore.listFrom(HadoopFileSystemLogStore.java:59)
	at org.apache.spark.sql.delta.storage.LogStoreAdaptor.listFrom(LogStore.scala:452)
	at org.apache.spark.sql.delta.storage.DelegatingLogStore.l